### Normalization of Points

As mentioned in Chapter 6 *Camera Calibration*, data normalization can improve estimation results.
In particular, this concerns the normalization before retrieving the projection matrix $\mathbf{P}$.
In this example, we script a small Python method that performs the normalization.

## Methods in Python
Let's first consider what methods are in Python, why they are important and how we define them...
A self-defined method is actually the same as the methods provided by the Python libraries (e.g. Numpy).
They have a name, they have input arguments and most of them have also output arguments.
They are comparable to math functions but they can also operate on non-math datatyes like strings (text).
  
For instance, `len` simply counts the number of elements (here: of a string) and `print` prints a string onto the console.

In [ ]:
text = "computer vision"
a = len(text)
print("The number of characters in '" + text + "' is " + str(a) + ".")

The definition of a method is always located before the first usage of this method. Now, let's define a very simply method that rounds a floating point number.
  
A method definition starts with the keyword `def` followed by the method name `my_own_round` and then in brackets the names of the input argument(s) `my_float`. After the brakets, there's always a colon.
The so-called body that consists of the instructions is indented. At the end, there is optionally a `return` statement that can return a value to the function call.

In [ ]:
# This is the method definition:
def my_own_round(my_float):
    # the decimals after the comma:
    decimals = my_float % 1 # % is the modulo operator
    integer_value = int(my_float) # the value before the comma
    
    # Let's see if we have to round up or down.
    if decimals >= 0.5:
        # round up
        result = integer_value + 1
    else:
        # round down = simply take the number without decimals
        result = integer_value
    
    # finally return the value of the result variable
    return result

# exemplary call:
# I want to share 5 pokemon cards with my best friend...
num_pokemons = 5
num_pokemons_for_my_friend = num_pokemons / 2
print("Before rounding:", num_pokemons_for_my_friend)

# Nooooo! Do not shred the cards. Let's round...
# method call:
num_pokemons_for_my_friend_rounded = my_own_round(num_pokemons_for_my_friend)
print("After rounding:", num_pokemons_for_my_friend_rounded)
# That's better ;)

The result of the method call was stored into the variable `num_pokemons_for_my_friend_rounded`.

## Back to Normalization
Now, we want to write a method that applies the normalization transformation of **Sec. 6.2.6** (pp. 9-10)
But first, let's define a 2D and a 3D structure as placeholders for world and image points...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# a tetragon in 2D projective space:
X_2D = np.array([
    [7, 2.5,   1, 2.5],
    [7, 2.5, 2.5,   1],
    [7,   1,    1,  1]
], dtype=np.float)

# a cube in 3D projective space:
X_3D = np.array([
    [1, 3, 1, 3, 1, 3, 1, 3],
    [1, 1, 3, 3, 1, 1, 3, 3],
    [1, 1, 1, 1, 3, 3, 3, 3],
    [1, 1, 1, 1, 1, 1, 1, 1]
], dtype=np.float)


# deactivate validation with validation = False
validation = True
debug = False

# simply ignore this method:
def validate(array, hash_val, error_msg):
    global validation
    if validation == False:
        return # do not validate
    if type(array) != np.ndarray:
        s = hash(array)
    else:
        s = 0
        for i in range(array.shape[0]):
            for j in range(array.shape[1]):
                s += hash(array[i, j] * 7*(i+1) + 13*(j+1))
    
    if debug == True:
        print("hash was:", s)
    assert s == hash_val, error_msg

The first method should build up the normalization homography from the scaling factor $s$ and the centroid $\tilde{\mathbf{L}}$:

$\mathbf{T} = \begin{bmatrix}
    s \cdot \mathbf{I}_{n\times n} & -s \cdot \tilde{\mathbf{L}} \\
    \mathbf{0}^T & 1
\end{bmatrix}$
  
whereas $\mathbf{I}_{n\times n}$ is an $n \times n$ identity matrix.
  
We can subdivide this into an affine part:
$\mathbf{A} = s \cdot \mathbf{I}_{n\times n}$, a translation part $\mathbf{T}' = -s \cdot \tilde{\mathbf{L}}$ and the last row $\begin{bmatrix}\mathbf{0}^T & 1 \end{bmatrix}$

In [ ]:
# assume centroid is an inhomogeneous vector of size nx1
def build_up_T(scale, centroid):
    n = # number of INHOMOGENEOUS dimensions # TODO: insert your code here
    assert n > 1
    
    affine_part = scale * np.eye(n)
    translation =  # TODO: insert your code here
    assert affine_part.shape[0] == affine_part.shape[1], "The shape of the affine part is not correct."
    
    # top part of size n x (n+1)
    top_part = np.hstack((        ,       )) # TODO: insert your code here
    assert top_part.shape[0] + 1 == top_part.shape[1], "The shape of the top part (affine & translation) is not correct."
    
    last_row_zeros = np.zeros((     ,     ), dtype=np.float) # TODO: insert your code here 
    assert last_row_zeros.shape[1] / last_row_zeros.shape[0] == n, "The part below the affine part has not the correct shape."
    last_row = np.hstack((last_row_zeros, np.array([[1]])))
    
    T = np.vstack((top_part, last_row))
    return T
    
# method test
test_scale = 0.5
test_centroid_2d = np.array([[1, 3]], dtype=np.float).T
test_centroid_3d = np.array([[1, 5, 3]], dtype=np.float).T

T_2d = build_up_T(test_scale, test_centroid_2d)
T_3d = build_up_T(test_scale, test_centroid_3d)
print(T_2d)

The next method should normalize the 2D and 3D structures.

In [ ]:
def normalize(my_points):
    n = # Now: n is number of HOMOGENEOUS dimensions (3 for 2D and 4 for 3D) # TODO: insert your code here 
    assert n >= 3 and n <= 4
    
    # convert my_points to inhomogeneous space
    my_points_inhom =  # TODO: insert your code here 
    validate(my_points.shape[0] / n * 7, 7, "The points are not correctly transformed to Cartesian coordiantes.")
    
    # find the inhomogeneous centroid of the 2D points...
    centroid = np.mean(      , axis=      , keepdims=True) # TODO: insert your code here 
    print("The centroid is now:")
    print(centroid)
    validate(centroid.shape[0] / (n-1) * 5, 5, "Shape of centroid is wrong.")
    
    # shift points to the origin s.th the centroid is at the origin
    my_points_inhom_shifted = my_points_inhom - centroid
    
    # calculate the Euclidean length of all vectors that point to the shifted 2D points...
    # For learning purposes, please avoid numpy's norm function.
    # You are allowed to use np.square and np.sqrt (square root).
    x_square = np.square(my_points_inhom_shifted[0:1, :])
    y_square = np.square(                               ) # TODO: insert your code here 
    euc_norm =       # TODO: insert your code here 
    print("The euclidean norm values for each point are:", euc_norm)
    validate(euc_norm.shape[0] * 1.2, 461168601842738689, "Shape of euc_norm is wrong. Please use ranges instead of indices to slice the point array.")
    
    # take the mean (one single number) of the Euclidean norm values 
    mean_norm = np.mean(         ) # TODO: insert your code here 
    print("The mean norm is:", mean_norm)
    validate(len(str(type(mean_norm))), 23, "The mean norm was not calculated properly. Is it one single number? No matrix...")
    
    # calculate the scale factor
    scale =           / mean_norm # TODO: insert your code here 
    validate(len(str(type(scale))), 23, "The scale value was not calculated properly. Is it one single number? No matrix...")
    
    # Now, let's use the previous method to build up the homography
    T = build_up_T(      ,        ) # TODO: insert your code here 
    
    # Lastly, we have to apply the normalization homography:
    my_points_normalized = T.dot(my_points)
    return my_points_normalized

# Some tests to ensure the method does what it is supposed to do...
print("---------- normalizing 2D points -----------")
X_2D_normalized = normalize(X_2D)
X_2D_normalized_inhom = X_2D_normalized[:2, :] / X_2D_normalized[2, :]
m2D = np.mean(np.linalg.norm(X_2D_normalized_inhom, axis=0))
assert m2D - np.sqrt(2) < 1e-10

print("---------- normalizing 3D points -----------")
X_3D_normalized = normalize(X_3D)
X_3D_normalized_inhom = X_3D_normalized[:2, :] / X_3D_normalized[2, :]
m3D = np.mean(np.linalg.norm(X_3D_normalized_inhom, axis=0))
assert m3D - np.sqrt(3) < 1e-10

print()
print("Congratulations! If you see this message printed onto the console you have done a good job!")

## Bonus
This sub task is not important for exam. It's just there for feeling like a hero afterwards...  
Task: Can you plot the 2D points with `matplotlib`? You might want to consider *Jupyter Notebooks* from other assignments...

In [ ]:
# TODO: insert your code here...